# 2a)

In [44]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from google.colab import files
uploaded = files.upload()

in_data = loadmat('face_emotion_data.mat')
# print([key for key in in_data]) # -- use this line to see the keys in the dictionary data structure

x_train = in_data['X']
y_train = in_data['y']

w = np.linalg.inv(x_train.transpose()@x_train)@x_train.transpose()@y_train
print("a)")
print(w)

y_hat = np.sign(x_train@w)
# x_train_barry = np.hstack([x_train[:,0],x_train[:,1],x_train[:,2]]).reshape(-1, 3)
x_train_barry = x_train[:,[0,2,3]]
w_barry = np.linalg.inv(x_train_barry.transpose()@x_train_barry)@x_train_barry.transpose()@y_train
y_hat_barry = np.sign(x_train_barry@w_barry)

error_vec = [0 if i[0]==i[1] else 1 for i in np.hstack((y_hat, y_train))]
error_vec_barry = [0 if i[0]==i[1] else 1 for i in np.hstack((y_hat_barry, y_train))]

print("e)")
print('Errors_using_all_nine_features: '+ str(sum(error_vec)/128))
print('Errors_using_three_features: '+ str(sum(error_vec_barry)/128))


num_subsets = 8
subset_size = 16
error_rates = []

# 進行八次交叉驗證
for i in range(num_subsets):
    # 定義訓練和測試集的索引
    test_indices = range(i * subset_size, (i + 1) * subset_size)
    train_indices = list(set(range(128)) - set(test_indices))
    # print(range)
    # print(test_indices)
    # print(train_indices)

    # 創建訓練和測試數據
    x_train_subset = x_train_barry[train_indices]
    # print(x_train_subset.shape)
    y_train_subset = y_train[train_indices]
    x_test_subset = x_train_barry[test_indices]
    y_test_subset = y_train[test_indices]

    # 訓練分類器
    w = np.linalg.inv(x_train_subset.T @ x_train_subset) @ x_train_subset.T @ y_train_subset

    # 進行預測
    y_hat = np.sign(x_test_subset @ w)

    # 計算錯誤數
    misclassifications = np.sum(y_hat.flatten() != y_test_subset.flatten())
    error_rate = misclassifications / subset_size

    error_rates.append(error_rate)

# 計算平均錯誤率
average_error_rate = np.mean(error_rates)

print("f)")
print('Error rates for each subset: ', error_rates)
print('Average error rate: ', average_error_rate)

Saving face_emotion_data.mat to face_emotion_data (36).mat
a)
[[ 0.94366942]
 [ 0.21373778]
 [ 0.26641775]
 [-0.39221373]
 [-0.00538552]
 [-0.01764687]
 [-0.16632809]
 [-0.0822838 ]
 [-0.16644364]]
e)
Errors_using_all_nine_features: 0.0234375
Errors_using_three_features: 0.0625
f)
Error rates for each subset:  [0.375, 0.0625, 0.0625, 0.0, 0.0, 0.0625, 0.0625, 0.0]
Average error rate:  0.078125


In [ ]:
plt.scatter(x_eval[:,0],x_eval[:,1], color=['c' if i==-1 else 'r' for i in y_eval[:,0]])
plt.title('eval data true class')
plt.show()

In [ ]:
## Classifier 1

# w = (X^T X)^(-1)X^T y
w_opt = np.linalg.inv(x_train.transpose()@x_train)@x_train.transpose()@y_train
y_hat = np.sign(x_eval@w_opt)

plt.scatter(x_eval[:,0],x_eval[:,1], color=['c' if i==-1 else 'r' for i in y_hat[:,0]])
plt.title('eval data predicted class (y_hat)')
plt.show()


In [ ]:
error_vec = [0 if i[0]==i[1] else 1 for i in np.hstack((y_hat, y_eval))]
plt.scatter(x_eval[:,0],x_eval[:,1], color=['c' if i==0 else 'r' for i in error_vec])
plt.title('errors')
plt.show()

print('Errors: '+ str(sum(error_vec)))

# 2b)

In [ ]:
## Classifier 2
x_train_2 = np.hstack((x_train**2, x_train, np.ones((n_train,1)) ))
x_eval_2 = np.hstack((x_eval**2, x_eval, np.ones((n_eval,1)) ))

w_opt_2 = np.linalg.inv(x_train_2.transpose()@x_train_2)@x_train_2.transpose()@y_train
y_hat_2 = np.sign(x_eval_2@w_opt_2)

plt.scatter(x_eval[:,0],x_eval[:,1], color=['c' if i==-1 else 'r' for i in y_hat_2[:,0]])
plt.title('predicted class on eval data')
plt.show()

In [ ]:
error_vec_2 = [0 if i[0]==i[1] else 1 for i in np.hstack((y_hat_2, y_eval))]
plt.scatter(x_eval[:,0],x_eval[:,1], color=['c' if i==0 else 'r' for i in error_vec_2])
plt.title('errors')
plt.show()

print('Error: '+ str(sum(error_vec_2)))

# 2c)

In [ ]:
## create new, correctly labeled points
n_new = 1000 #number of new datapoints
x_train_new = np.hstack((np.zeros((n_new,1)), 3*np.ones((n_new,1))))
y_train_new = np.ones((n_new,1))

## add these to the training data
x_train_outlier = np.vstack((x_train,x_train_new))
y_train_outlier = np.vstack((y_train,y_train_new))
plt.scatter(x_train_outlier[:,0],x_train_outlier[:,1], color=['c' if i==-1 else 'r' for i in y_train_outlier[:,0]])
plt.title('new training data')
plt.show()

In [ ]:
#train with new data
w_opt_outlier = np.linalg.inv(x_train_outlier.transpose()@x_train_outlier)@x_train_outlier.transpose()@y_train_outlier
y_hat_outlier = np.sign(x_eval@w_opt_outlier)

plt.scatter(x_eval[:,0],x_eval[:,1], color=['c' if i==-1 else 'r' for i in y_hat_outlier[:,0]])
plt.title('predicted class on eval data')
plt.show()

In [ ]:
error_vec = [0 if i[0]==i[1] else 1 for i in np.hstack((y_hat_outlier, y_eval))]
plt.scatter(x_eval[:,0],x_eval[:,1], color=['c' if i==0 else 'r' for i in error_vec])
plt.title('errors')
plt.show()

print('Errors: '+ str(sum(error_vec)))